In [1]:
!pip install transformers sentencepiece torch bert-score nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import pandas as pd

df = pd.read_csv("/content/test_dataset_cleaned.csv", encoding="utf-8")
df.head()

,Index,Input,Output
0,0,Summarize: You okay now kind of taking a look ...,The patient loves their job and has been very ...
1,1,Summarize: Are you doing today? Good. How you ...,The patient has had to give weekly presentatio...
2,2,Summarize: How are you? Not well? No what's go...,The patient has been at their job for 20 years...
3,3,"Summarize: So, alright, let's take a step back...",The patient has been concerned about the work ...
4,4,Summarize: Tell me about sure for any spiritua...,The patient was raised Catholic and they feel ...


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load Pegasus model
model_name = "google/pegasus-large" 
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
!pip install transformers sentencepiece torch bert-score nltk

device = "cuda"


df = pd.read_csv("/content/test_dataset_cleaned.csv", encoding="utf-8")
df.head()

# Load Pegasus model
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

if device == "cuda":
  model = model.to(device)

def generate_summary(text):
    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt", max_length=512).to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=100, num_beams=5, length_penalty=2.0)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

df["Generated_Summary"] = df["Input"].apply(generate_summary)

df.to_csv("pegasus_summarized_dataset.csv", index=False)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
df

,Index,Input,Output,Generated_Summary
0,0,Summarize: You okay now kind of taking a look ...,The patient loves their job and has been very ...,"I wasn't having the best time of my life, but ..."
1,1,Summarize: Are you doing today? Good. How you ...,The patient has had to give weekly presentatio...,"Yeah, probably for the last eight months, mayb..."
2,2,Summarize: How are you? Not well? No what's go...,The patient has been at their job for 20 years...,"And because it was kind of more sudden, we did..."
3,3,"Summarize: So, alright, let's take a step back...",The patient has been concerned about the work ...,"Yeah, I have a supervisor so they check up on ..."
4,4,Summarize: Tell me about sure for any spiritua...,The patient was raised Catholic and they feel ...,"Like, if you do this, you're gonna go to hell ..."
5,5,"Summarize: Okay, I understand you've been havi...",The patient has been sleeping excessively and ...,"I'm really tired You know, I I'm tired when I ..."
6,6,Summarize: Afternoon. I'm glad you could get i...,The patient feels anxious. The patient hasn't ...,"Well, I looked at the gym and there weren't re..."
7,7,"Summarize: In one day, and none of them worked...",The patient is frustrated that they are not ab...,"Summarize: In one day, and none of them worked..."
8,8,Summarize: Hi You How you doing today. Good. H...,The patient's brother expressed concerns about...,It's been great like I've I mean I used to hav...
9,9,Summarize: Alisa How you doing today? Good? Ho...,The patient has been experiencing anxiety. The...,"Um, we have our daily meetings, and I actually..."


In [9]:
import nltk
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from bert_score import score
import numpy as np

df = pd.read_csv("pegasus_summarized_dataset.csv")

def compute_sentence_bleu(reference, hypothesis):
    reference_tokens = [nltk.word_tokenize(reference)] 
    hypothesis_tokens = nltk.word_tokenize(hypothesis) 
    smoothing = SmoothingFunction().method4
    return sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=smoothing)

# Compute BLEU scores
df["BLEU_Score"] = df.apply(lambda row: compute_sentence_bleu(row["Output"], row["Generated_Summary"]), axis=1)

df["BLEU_Score"] = (df["BLEU_Score"] - df["BLEU_Score"].min()) / (df["BLEU_Score"].max() - df["BLEU_Score"].min())

# Compute Corpus BLEU
references = [[nltk.word_tokenize(ref)] for ref in df["Output"].tolist()]
hypotheses = [nltk.word_tokenize(hyp) for hyp in df["Generated_Summary"].tolist()]
final_bleu = corpus_bleu(references, hypotheses, smoothing_function=SmoothingFunction().method4)

final_bleu = (final_bleu - 0) / (1 - 0)

# Compute BERTScore
P, R, F1 = score(df["Generated_Summary"].astype(str).tolist(), df["Output"].astype(str).tolist(), lang="en", rescale_with_baseline=True)

df["BERTScore_F1"] = (F1 - np.min(F1.numpy())) / (np.max(F1.numpy()) - np.min(F1.numpy()))
final_bert = df["BERTScore_F1"].mean()

df.to_csv("evaluation_results_pegasus.csv", index=False)

print(f"Final Corpus BLEU Score (Scaled): {final_bleu:.4f}")
print(f"Final BERTScore F1 (Scaled): {final_bert:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Final Corpus BLEU Score (Scaled): 0.0247
Final BERTScore F1 (Scaled): 0.3015
